In [8]:
import numpy as np
import scipy.sparse as sps

# Create classes: Perceptron, Connection

# Create function: SIGN

In [9]:
# Parameters
n_inputs = 2
hidden_size = 1
n_hidden = 1

In [123]:
class Network(object): 
    def __init__(self, n_inputs, hidden_size, n_hidden_layers, learning_rate = 0.2):
        self.n_inputs = n_inputs
        self.input = []
        self.hidden_size = hidden_size
        self.n_hidden = n_hidden
        self.n_nodes = n_inputs + n_hidden*hidden_size + 1 #Number of nodes in network
        self.connections = np.array([[0 for x in range(self.n_nodes)] for y in range(self.n_nodes)], dtype = float )
        self.nodes = [0]*self.n_nodes
        self.lr = learning_rate
        
        # Initiate input nodes
        for i in range(n_inputs): 
            self.nodes[i] = Node(i)
            
        # Initiate hidden nodes
        #for i in range(n_hidden_layers): 
            
        # Initiate output node
        self.nodes[-1] = (Perceptron("SIGN"))
        
        # Set weights of connections
        for i in range(self.n_inputs): 
            self.connections[i][-1] = 1; # From input to perceptron: weight = 1. 
            
        self.connections[0][-1] = 3/2
        
        # Set w0 of perceptron
        self.connections[-1][-1] = -2
        print(self.connections)
            
    def test(self, inputs):
        # Takes input and returns output according to the network. 
        output = 0
        if len(inputs) != self.n_inputs:
            print("Wrong number of inputs %s, should be %s" %(inputs.length, self.n_inputs))
        
        for i, inp in enumerate(inputs):
            self.nodes[i].setValue(inp)
        
        return self.nodes[-1].output(self.connections, self.nodes)
    
    def fit(self, epochs, input_data, output_data):
        pass
        # Fits the models according to the input and output data. 
        # Input is expected as an e * i matrix, output expected as e * 1

In [124]:
class Node(object):
    def __init__(self, index, value = None):
        self.value = value
        self.index = index 
        
    def output(self, connections = None, nodes = None): 
        # for each node in input
        return self.value  

    def setValue(self, value):
        self.value = value
    
class Perceptron(Node): 
    # Has a bunch of input-connections and output connections
    def __init__(self, value = None, activation_function = "SIGN"): 
        Node.__init__(self, value)
        self.activation_function = activation_function
        
    def sign(self, inp): 
        if inp < 0:
            return -1
        elif inp > 0: 
            return 1
        else: 
            return 0
        
    def output(self, connections, nodes):
        output = 0
        for i, weight in enumerate(connections[:-1,2]): # Recursive stuff
            output += nodes[i].output(connections, nodes)*weight
            
        
        output += 1*connections[-1][-1] # Add bias. 
        
        return self.sign(output)
        

In [127]:
n = Network(2,0,0)

print(n.test([1,0]))



[[ 0.   0.   1.5]
 [ 0.   0.   1. ]
 [ 0.   0.  -2. ]]
-1
